In [ ]:
import json
import io
import folium
import matplotlib
import matplotlib.pyplot as plt
import osmnx as ox 
import numpy as np
from folium.features import DivIcon
from PIL import Image

In [ ]:
with open('../data/matched-results-1500.json', 'r') as f:
    match_results = json.load(f)
print(len(match_results))

In [ ]:
colors = plt.cm.jet(np.linspace(0, 1, 15))

In [ ]:
initial_location = [41.1579, -8.6291]  # Example coordinates for Porto, Portugal
fmap = folium.Map(location=initial_location, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(fmap)

for i in range(15):
    gps_points = match_results[i]['POLYLINE']
    gps_points = [p[::-1] for p in gps_points]

    matched_geometry = match_results[i]['MATCHED_RESULTS']['Matched_geometry']
    mgeom = [i[::-1] for i in matched_geometry] 

    hex_color = matplotlib.colors.to_hex(colors[i])
    
    folium.PolyLine(mgeom, color=hex_color, weight=3).add_to(fmap)
    
    for point in gps_points:
        folium.CircleMarker(location=point, radius=2, color=hex_color).add_to(fmap)

fmap.save('../data/matched-15-routes-folium.html')

#img_data = fmap._to_png(5)
#img = Image.open(io.BytesIO(img_data))
#img.save('../data/matched-15-routes-folium.png')

#fmap

In [ ]:
for i in range(15):
    gps_points = match_results[i]['POLYLINE']
    gps_points = [p[::-1] for p in gps_points]

    latitudes, longitudes = zip(*gps_points)
    centroid = [np.mean(latitudes), np.mean(longitudes)]

    matched_geometry = match_results[i]['MATCHED_RESULTS']['Matched_geometry']
    mgeom = [i[::-1] for i in matched_geometry] 

    hex_color = matplotlib.colors.to_hex(colors[i])

    fmap = folium.Map(location=centroid, zoom_start=15)
    folium.TileLayer('cartodbpositron').add_to(fmap)

    folium.PolyLine(mgeom, color=hex_color, weight=2).add_to(fmap)

    for j, point in enumerate(gps_points):
        folium.CircleMarker(location=point, radius=1, color=hex_color).add_to(fmap)
        folium.map.Marker(
            #location=[point[0]+0.0005, point[1]],
            location=point,
            icon=DivIcon(icon_anchor=(5, 20), html=f'<div style="font-size: 10pt">%s</div>' % str(j+1)),
        ).add_to(fmap)

    # Save each map as an HTML file
    fmap.save(f'../data/matched-route-{i}-folium.html')

    #img_data = fmap._to_png(5)
    #img = Image.open(io.BytesIO(img_data))
    #img.save(f'../data/matched-route-{i}-folium.png')

In [ ]:
i = 3

gps_points = match_results[i]['POLYLINE']
gps_points = [p[::-1] for p in gps_points]
print(len(gps_points))

latitudes, longitudes = zip(*gps_points)
centroid = [(max(latitudes)+min(latitudes))/2, (max(longitudes)+min(longitudes))/2]
print(centroid)

matched_geometry = match_results[i]['MATCHED_RESULTS']['Matched_geometry']
mgeom = [i[::-1] for i in matched_geometry] 
print(len(mgeom))

hex_color = matplotlib.colors.to_hex(colors[i])

In [ ]:
_gps_points = gps_points[:18]
_mgeom = mgeom[:244]

fmap = folium.Map(location=centroid, zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(fmap)

folium.PolyLine(_mgeom, color=hex_color, weight=2).add_to(fmap)

for j, point in enumerate(_gps_points, 0):
    folium.CircleMarker(location=point, radius=1, color=hex_color).add_to(fmap)
    folium.map.Marker(
        location=point,
        icon=DivIcon(icon_anchor=(5, 20), html=f'<div style="font-size: 10pt">%s</div>' % str(j+1)),
    ).add_to(fmap)

#img_data = fmap._to_png(5)
#img = Image.open(io.BytesIO(img_data))
#img.save(f'../data/matched-route-{i}-folium-1.png')
    
fmap


In [ ]:
_gps_points = gps_points[17:]
_mgeom = mgeom[242:]

fmap = folium.Map(location=centroid, zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(fmap)

folium.PolyLine(_mgeom, color=hex_color, weight=2).add_to(fmap)

for j, point in enumerate(_gps_points, 17):
    folium.CircleMarker(location=point, radius=1, color=hex_color).add_to(fmap)
    folium.map.Marker(
        location=point,
        icon=DivIcon(icon_anchor=(5, 20), html=f'<div style="font-size: 10pt">%s</div>' % str(j+1)),
    ).add_to(fmap)
    
#img_data = fmap._to_png(5)
#img = Image.open(io.BytesIO(img_data))
#img.save(f'../data/matched-route-{i}-folium-2.png')

fmap